In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Dheer\\OneDrive\\Desktop\\MLOps\\MediWatch_project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Dheer\\OneDrive\\Desktop\\MLOps\\MediWatch_project'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    n_estimators: float
    max_depth: float
    learning_rate: float
    target_column: str

In [6]:
from Mediwatch_project.constants import *
from Mediwatch_project.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.model_params
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            n_estimators = params.n_estimators,
            max_depth = params.max_depth,
            learning_rate = params.learning_rate,
            target_column = schema.name
        )
        return model_trainer_config

In [8]:
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd
import os
from Mediwatch_project import logger
import joblib
import xgboost as xgb
import numpy as np
import sklearn

In [ ]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        # Load training and testing data
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        # Split features and target
        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[self.config.target_column]
        test_y = test_data[self.config.target_column]

        # Handle categorical columns
        for col in train_x.select_dtypes(['category']).columns:
            train_x[col] = train_x[col].cat.codes
            test_x[col] = test_x[col].cat.codes

        # Handle class imbalance
        neg, pos = np.bincount(train_y)
        scale_pos_weight = neg / pos

        # Define XGBoost classifier using parameters from config
        xgb_clf = xgb.XGBClassifier(
            objective='binary:logistic',
            eval_metric='logloss',
            scale_pos_weight=scale_pos_weight,
            n_estimators=self.config.n_estimators,
            max_depth=self.config.max_depth,
            learning_rate=self.config.learning_rate
        )

        # Train the model
        xgb_clf.fit(train_x, train_y)

        # Save the trained model
        joblib.dump(xgb_clf, os.path.join(self.config.root_dir, self.config.model_name))


In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2025-08-07 19:26:00,914: INFO: common: Raw YAML content from WindowsPath('config/config.yaml'): {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://raw.githubusercontent.com/dheeraj24797/MediWatch_project/main/diabetic_data.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'data_validation': {'root_dir': 'artifacts/data_validation', 'unzip_data_dir': 'artifacts/data_ingestion/diabetic_data.csv', 'STATUS_FILE': 'artifacts/data_validation/status.txt'}, 'data_transformation': {'root_dir': 'artifacts/data_transformation', 'data_path': 'artifacts/data_ingestion/diabetic_data.csv'}, 'model_trainer': {'root_dir': 'artifacts/model_trainer', 'train_data_path': 'artifacts/data_transformation/train.csv', 'test_data_path': 'artifacts/data_transformation/test.csv', 'model_name': 'model.joblib'}} (type=<class 'dict'>)]
[2025-08-07 19:26:00,919: INFO: common: YAML file: config\config.y

c:\Users\Dheer\OneDrive\Desktop\MLOps\MediWatch_project\env\lib\site-packages\xgboost\core.py:158: UserWarning: [19:26:03] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
